In [ ]:
!pip install numpy pandas plotly --quiet

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# Parameters
initial_capital = 100
win_rate = 0.1          # 10% chance to win
risk_per_trade = 0.01   # 1% risk per trade
lot_size = initial_capital * risk_per_trade
n_simulations = 1000    # number of Monte Carlo runs

# Function to simulate one run until account goes to zero
def simulate_run(initial_capital, win_rate, lot_size):
    capital = initial_capital
    attempts = 0
    history = [capital]
    while capital > 0:
        attempts += 1
        outcome = np.random.rand() < win_rate
        if outcome:
            capital += lot_size
        else:
            capital -= lot_size
        history.append(capital)
    return attempts, history

# Run multiple simulations
results = []
histories = []
for _ in range(n_simulations):
    attempts, history = simulate_run(initial_capital, win_rate, lot_size)
    results.append(attempts)
    histories.append(history)

# Create DataFrame of attempts
df = pd.DataFrame(results, columns=['Attempts'])

# Histogram of attempts before ruin
fig_hist = px.histogram(df, x='Attempts', nbins=50,
                        title='Distribution of Attempts Before Account Goes to Zero',
                        labels={'Attempts':'Number of Attempts'})
fig_hist.show()

# --- Find the peak of the distribution ---
mode_attempts = df['Attempts'].mode()[0]   # most frequent attempt count
count_mode = (df['Attempts'] == mode_attempts).sum()
print(f"Peak of distribution: {mode_attempts} attempts (occurred {count_mode} times)")

# Select all runs that ended at this mode attempt count
mode_runs = [hist for hist, att in zip(histories, results) if att == mode_attempts]

# Build DataFrame for plotting
curve_data = []
for i, hist in enumerate(mode_runs, 1):
    curve_data.extend([{'Attempt': j, 'Capital': c, 'Run': f'Run {i}'} for j, c in enumerate(hist)])

df_curves = pd.DataFrame(curve_data)

# Plot equity curves for mode runs
fig_curves = px.line(df_curves, x='Attempt', y='Capital', color='Run',
                     title=f'Equity Curves for Peak Distribution ({mode_attempts} Attempts, {count_mode} runs)',
                     labels={'Capital':'Account Balance'})
fig_curves.show()